In [1]:
import numpy as np
from numpy import linalg
import os
import json

In [2]:
def gaussian_entropy(cov_matrix):
    cov_matrix = np.atleast_2d(cov_matrix)
    n = len(cov_matrix)
    det_cov = linalg.det(cov_matrix)
    return 1/2 * np.log2((2*np.pi*np.exp(1))**n * det_cov)

def teoric_TC(cov_matrix):
    n = len(cov_matrix)
    # H(X1,...,Xn)
    H_x = gaussian_entropy(cov_matrix)
    # H(Xi)
    H_xi = [gaussian_entropy(cov_matrix[i, i]) for i in range(n)]
    # sum(H(Xi))
    sum_H_xi = np.sum(H_xi)
    # TC
    TC = sum_H_xi - H_x
    return TC

def teoric_DTC(cov_matrix):
    n = len(cov_matrix)
    # H(X1,...,Xi-1,Xi+1,...,Xn)
    H_xis_ = [gaussian_entropy(np.delete(np.delete(cov_matrix, j, axis=1), j, axis=0)) for j in range(n)] 
    # sum(H(X1,...,Xi-1,Xi+1,...,Xn))
    sum_H_xi_ = np.sum(H_xis_) 
    # H(X1,...,Xn)
    H_x = gaussian_entropy(cov_matrix)
    # (1-n) * H(X1,...,Xn)
    prod_H_x = (1-n) * H_x
    # DTC
    DTC = prod_H_x + sum_H_xi_
    return DTC

def teoric_oinfo(cov_matrix):
    oinfo = teoric_TC(cov_matrix) - teoric_DTC(cov_matrix)
    return oinfo

def info_mutua(cov, columns1, columns2):
    marginal1 = cov[columns1][:,columns1]
    marginal2 = cov[columns2][:,columns2]
    columns_joint = [*columns1, *columns2]
    columns_joint.sort()
    joint = cov[columns_joint][:,columns_joint]
    I = gaussian_entropy(marginal1) + gaussian_entropy(marginal2) - gaussian_entropy(joint)
    return I

In [3]:
def teoric_TC2(cov_matrix):
    N = len(cov_matrix)
    terms = []
    for i in range(1,N):
        terms.append(info_mutua(cov_matrix, [i], np.arange(i)))
    return np.sum(terms), terms

def teoric_DTC2(cov_matrix):
    N = len(cov_matrix)
    terms = [info_mutua(cov_matrix, [N-1], np.arange(N-1))]
    for j in range(1,N-1):
        info1 = info_mutua(cov_matrix, [j], [*np.arange(j), *np.arange(j+1, N)])
        info2 = info_mutua(cov_matrix, [j], np.arange(j+1, N))
        info_cond = info1 - info2
        terms.append(info_cond)
    return np.sum(terms), terms

def teoric_oinfo2(cov_matrix):
    oinfo = teoric_TC(cov_matrix) - teoric_DTC(cov_matrix)
    return oinfo

In [4]:
a = np.arange(10)
a

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [5]:
a[:10]

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [6]:
j = 2
N = 10
[*np.arange(j), *np.arange(j+1, N)]

[0, 1, 3, 4, 5, 6, 7, 8, 9]

In [7]:
def makeSamples(parametersList, repetition_number, save_npy=False):    
    i=1
    for parameters in parametersList:
        os.mkdir('./muestras/gaussian_'+str(i))
        cov = parameters['cov']
        dim = len(cov)
        TC_teoric = teoric_TC(cov)
        DTC_teoric = teoric_DTC(cov)
        O_teoric = TC_teoric-DTC_teoric
        header = f" {parameters['n_samples']} samples \n Cov = \n{cov}\n TC_teoric = {TC_teoric}\n DTC_teoric = {DTC_teoric}\n O_teoric = {O_teoric}"
        for j in range(1,repetition_number+1):
            np.random.seed(j)
            normal = np.random.multivariate_normal(np.zeros(dim), parameters['cov'], parameters['n_samples'])
            if (save_npy):
                np.save(f'./muestras/gaussian_{i}/samples{j}.npy',normal)
            else:
                np.savetxt(f'./muestras/gaussian_{i}/samples{j}.txt',normal, header=header, comments='%')
        i+=1

In [8]:
def makeJson(parametersList):
    gaussians={}
    for i, parameters in enumerate(parametersList):
        cov = parameters['cov']
        n_samples = parameters['n_samples']
        TC_teoric_value, TC_teoric_list = teoric_TC2(cov)
        DTC_teoric_value, DTC_teoric_list = teoric_DTC2(cov)
        O_teoric_value = TC_teoric_value-DTC_teoric_value
        gaussians[f"gaussian_{i+1}"] = {
            "cov": cov.tolist(),
            "TC": TC_teoric_value,
            "DTC": DTC_teoric_value,
            "O": O_teoric_value,
            "TC_list": TC_teoric_list,
            "DTC_list": DTC_teoric_list
        }
    with open("./muestras/index.json", "w") as f:
        json.dump(gaussians, f)

In [9]:
def crearMatriz(N, seed = 1):
    np.random.seed(seed)
    covmat = np.random.rand(N,N)
    covmat = covmat @ covmat.transpose()
    return covmat

In [10]:
"""for i in range(1000):
    cov=crearMatriz(10,seed=i)
    o = teoric_oinfo(cov)
    if abs(o)<0.01:
        print(o,i)"""

'for i in range(1000):\n    cov=crearMatriz(10,seed=i)\n    o = teoric_oinfo(cov)\n    if abs(o)<0.01:\n        print(o,i)'

In [11]:
n_samples = 1000000
parametersList = [
{
    "n_samples": n_samples,
    "cov": crearMatriz(3, seed = 98)#Negativo
},
{
    "n_samples": n_samples,
    "cov": crearMatriz(3, seed = 746)#Positivo
},
{
    "n_samples": n_samples,
    "cov": crearMatriz(3, seed = 331)#Cercano a cero
},
{
    "n_samples": n_samples,
    "cov": crearMatriz(4, seed = 222)#Negativo
},
{
    "n_samples": n_samples,
    "cov": crearMatriz(4, seed = 569)#Positivo
},
{
    "n_samples": n_samples,
    "cov": crearMatriz(4, seed = 359)#Cercano a cero
},
{
    "n_samples": n_samples,
    "cov": crearMatriz(10, seed = 33)#Negativo
},
{
    "n_samples": n_samples,
    "cov": crearMatriz(10, seed = 24)#Positivo
},
{
    "n_samples": n_samples,
    "cov": crearMatriz(10, seed = 775)#Cercano a cero
}
]

In [12]:
for parameters in parametersList:
    print(np.linalg.det(parameters['cov']))

3.5047283257553657e-10
0.0009908436809163697
0.1884437653096859
9.359386841832904e-07
5.2467902032759716e-05
0.000886192822186952
5.0394389847273145e-06
0.016170695726148265
0.00041687596142928876


In [14]:
#os.mkdir('./muestras')
makeJson(parametersList)
#makeSamples(parametersList,100,save_npy=False)
#makeSamples(parametersList,100,save_npy=True)

In [ ]:
np.arange(3)

In [ ]:
np.arange(0)